In [1]:
from loader import load_contract, load_dataset
from schema_builder import build_schema, validate_schema    

In [2]:
df = load_dataset('C:\\Users\\Kevin\\OneDrive\\Personal\\GitHub\\data-contracts-data-profiler\\Data_Profiler_Data_Contracts\\data\\mock_data.csv')
contract = load_contract('C:\\Users\\Kevin\\OneDrive\\Personal\\GitHub\\data-contracts-data-profiler\\Data_Profiler_Data_Contracts\\contract\\data_contract.yaml')


In [3]:
print(build_schema(contract))

<Schema DataFrameSchema(
    columns={
        'id': <Schema Column(name=id, type=DataType(str))>
        'first_name': <Schema Column(name=first_name, type=DataType(str))>
        'last_name': <Schema Column(name=last_name, type=DataType(str))>
        'gender': <Schema Column(name=gender, type=DataType(str))>
        'ip_address': <Schema Column(name=ip_address, type=DataType(str))>
        'weight': <Schema Column(name=weight, type=DataType(float64))>
        'age': <Schema Column(name=age, type=DataType(str))>
        'salary': <Schema Column(name=salary, type=DataType(float64))>
        'country': <Schema Column(name=country, type=DataType(str))>
        'monthly_savings': <Schema Column(name=monthly_savings, type=DataType(float64))>
        'marital_status': <Schema Column(name=marital_status, type=DataType(str))>
        'role': <Schema Column(name=role, type=DataType(str))>
        'height': <Schema Column(name=height, type=DataType(float64))>
        'education_level': <Schema

In [4]:
validate_schema(df, contract)

[{'rule': 'schema_check',
  'column': 'id',
  'check': "dtype('str')",
  'failed_value': 1,
  'error': None,
  'severity': 'high',
  'severity_points': 5,
  'message': "dtype('str') failed on column id: 1",
  'timestamp': '2025-05-10T13:06:08.636574'},
 {'rule': 'schema_check',
  'column': 'years_experience',
  'check': "dtype('str')",
  'failed_value': 179,
  'error': None,
  'severity': 'high',
  'severity_points': 5,
  'message': "dtype('str') failed on column years_experience: 179",
  'timestamp': '2025-05-10T13:06:08.636574'},
 {'rule': 'schema_check',
  'column': 'age',
  'check': "dtype('str')",
  'failed_value': 29,
  'error': None,
  'severity': 'high',
  'severity_points': 5,
  'message': "dtype('str') failed on column age: 29",
  'timestamp': '2025-05-10T13:06:08.636574'},
 {'rule': 'schema_check',
  'column': 'age',
  'check': "dtype('str')",
  'failed_value': 72,
  'error': None,
  'severity': 'high',
  'severity_points': 5,
  'message': "dtype('str') failed on column age: